In [1]:
%%time

import joblib, time, os, copy, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision

import torch.cuda as cutorch

#enable it when running in google cloud, to upload/download file from/to VM to/from google cloud bucket
from google.cloud import storage

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#turn on cpu for troubleshooting as gpu doesnt throw proper error message, runing in cpu shows more specific error message
#device = torch.device("cpu") 
print(device)
for i in range(cutorch.device_count()):
    print(torch.cuda.get_device_properties("cuda:" + str(i)))

bucket_root_path = "weicheng30417"
project_data_folder = "data/breakfast-img-data/"
torch.manual_seed(0)

cuda:0
_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)
CPU times: user 432 ms, sys: 312 ms, total: 744 ms
Wall time: 805 ms


In [2]:
def download_all_data_from_bucket():
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    file_id = project_data_folder
    blobs=bucket.list_blobs(prefix=file_id)
    for blob in blobs:        
        file_names = blob.name.split("/")
        if len(file_names) == 3:
            if file_names[2] != "":
                file_name = file_names[2]            
                blob.download_to_filename(file_name)
                print("Download from {0} to local {1}".format(blob.name, file_name))
                

def download_file_from_bucket(file):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    butcketFile = project_data_folder + file
    blob = bucket.blob(butcketFile)
    blob.download_to_filename(file)
    print("Download from {0} to local {1}".format(butcketFile, file))
    
def upload_all_files():    
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    
    directory = os.getcwd()
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv") or filename.endswith(".ipynb") or filename.endswith(".model"):             
            butcketFile = project_data_folder + file
            blob = bucket.blob(butcketFile)
            blob.upload_from_filename(filename)
            print("Upload from local {0} to {1}".format(filename, butcketFile))
            
def upload_files(files):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    for file in files:
        butcketFile = project_data_folder + file
        blob = bucket.blob(butcketFile)
        blob.upload_from_filename(file)
        print("Upload from local {0} to {1}".format(file, butcketFile))

def flatten(ls):
     for item in ls:
            for child in list(item):
                yield child

def get_segment_positions(x):
    ps = x.segment.split()
    ls = []
    for i in range(len(ps)-1):
        if i == 0:
            ls.append([int(ps[i]), int(ps[i+1])])
        else:
            ls.append([int(ps[i])+1, int(ps[i+1])])        
    return ls

def get_segment_features(x):
    ls = []
    for rg in x.positions:
        ls.append(x.feature[rg[0]:rg[1]])            
    return ls

def splitDataFrameList(df,target_column):
    def splitListToRows(row,row_accumulator,target_column):
        split_row = row[target_column]
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column))
    new_df = pd.DataFrame(new_rows)
    return new_df
                
def get_train_data():
    training_segment = pd.read_csv('training_segment.txt', header=None, names = ['segment'])
    training_segment['feature'] = joblib.load('train_feature.joblib')
    training_segment['positions'] = training_segment.apply(lambda x: get_segment_positions(x), axis=1)
    training_segment['feature'] = training_segment.apply(lambda x: get_segment_features(x), axis=1)
    training_segment = splitDataFrameList(training_segment, 'feature')
    training_segment['label'] = list(flatten(joblib.load('train_label.joblib')))
    training_segment = training_segment.drop(['segment','positions'], axis = 1)
    return training_segment

def get_test_data():
    test_segment = pd.read_csv('test_segment.txt', header=None, names = ['segment'])
    test_segment['feature'] = joblib.load('test_feature.joblib')
    test_segment['positions'] = test_segment.apply(lambda x: get_segment_positions(x), axis=1)
    test_segment['feature'] = test_segment.apply(lambda x: get_segment_features(x), axis=1)
    test_segment = splitDataFrameList(test_segment, 'feature')
    test_segment['ID'] = test_segment.index
    test_segment = test_segment.drop(['segment','positions'], axis = 1)
    return test_segment

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        ms = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)           

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)            
            print('{} Loss: {:.4f}, {} Acc: {:.4f}'.format(phase, epoch_loss, phase, epoch_acc))
        
        time_taken = str(datetime.timedelta(seconds=time.time() - ms))
        print('time taken: {}'.format(time_taken))
            

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, val_acc_history

def extract_or_pad(x):    
    target = torch.zeros([400, 400], dtype=torch.float64)
    w = x.shape[0]
    
    if w <= 400:       
        no_tile = (400 // w) + 1        
        x = torch.cat(no_tile*[x])
        
    target = x[:400, :400]
    return target.view(1, 400, 400)

In [3]:
%%time
train_df = get_train_data()
print(train_df.info())
print(train_df.feature[0].shape)
print(train_df.feature[1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7075 entries, 0 to 7074
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  7075 non-null   object
 1   label    7075 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 110.7+ KB
None
torch.Size([260, 400])
tensor([[-16.2783,   4.9215,  -3.6278,  ...,  -6.3865,  -1.0839,  -6.3627],
        [-19.2205,   6.1413,  -4.5396,  ...,  -5.2037,  -2.6973,  -4.9110],
        [-19.7272,   7.0195,  -4.0093,  ...,  -4.2315,  -7.2786,  -6.9802],
        ...,
        [-16.5469,   6.7402,  -2.2102,  ...,  -6.2332,  -6.7890,  -5.9929],
        [-18.1927,   5.1798,  -2.2626,  ...,  -5.0009,  -4.8182,  -4.7292],
        [-19.1877,   5.8449,  -4.2045,  ...,  -4.2093,  -6.4421,  -7.3169]],
       dtype=torch.float64)
CPU times: user 2.31 s, sys: 9.7 s, total: 12 s
Wall time: 12 s


In [4]:
%%time
train_data = train_df.sample(frac=1.0)
train_data['label'] = train_data.apply(lambda x : x["label"] - 1, axis = 1)
train_data['feature'] = train_data.apply(lambda x : extract_or_pad(x.feature), axis = 1)
print(train_data.info())
print(train_data.feature[1].shape)
print(train_data.feature[1])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7075 entries, 3803 to 4021
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  7075 non-null   object
 1   label    7075 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 165.8+ KB
None
torch.Size([1, 400, 400])
tensor([[[-16.2783,   4.9215,  -3.6278,  ...,  -6.3865,  -1.0839,  -6.3627],
         [-19.2205,   6.1413,  -4.5396,  ...,  -5.2037,  -2.6973,  -4.9110],
         [-19.7272,   7.0195,  -4.0093,  ...,  -4.2315,  -7.2786,  -6.9802],
         ...,
         [-14.9790,   2.9633,  -1.4342,  ...,  -4.2220,  -7.2918, -10.0969],
         [-16.3594,   4.0857,  -1.0715,  ...,  -5.6668,  -9.0479,  -9.3844],
         [-15.0685,   5.8967,  -1.7798,  ...,  -5.0414,  -8.3543,  -8.0798]]],
       dtype=torch.float64)
CPU times: user 24.1 s, sys: 4.12 s, total: 28.3 s
Wall time: 5.74 s


In [5]:
%%time
num_classes = 47
batch_size = 4

train_feature = torch.stack(train_data['feature'].tolist())
train_label = torch.tensor(train_data['label'].values.astype(np.long))

train_dataset = torch.utils.data.TensorDataset(train_feature, train_label)

train_size = int(0.8 * len(train_data))
val_size = int(0.1 * len(train_data))
test_size = len(train_data) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

dataloaders_dict = {'train':train_loader, 'val':val_loader}
print("len(train_loader.dataset) = ", len(train_loader.dataset))
print("len(val_loader.dataset) = ", len(val_loader.dataset))
print("len(test_loader.dataset) = ", len(test_loader.dataset))

len(train_loader.dataset) =  5660
len(val_loader.dataset) =  707
len(test_loader.dataset) =  708
CPU times: user 2.22 s, sys: 3.83 s, total: 6.05 s
Wall time: 5.29 s


In [6]:
from torch.nn import *
from torch.nn.functional import *

class MyNet(Module):
    def __init__(self, batch_size, num_classes):
        super(MyNet, self).__init__()
        self.batch_size = batch_size
        self.num_classes = num_classes        
        
        self.conv1 = Conv2d(in_channels=1, out_channels=batch_size, kernel_size=3, padding=1)
        self.conv1_bn = BatchNorm2d(num_features=batch_size)
        
        i=0
        self.conv2 = Conv2d(in_channels=batch_size*pow(2,i), out_channels=batch_size*pow(2,i+1), kernel_size=3, padding=1)
        self.conv2_bn = BatchNorm2d(num_features=batch_size*pow(2,i+1))
        i = i + 1
        
        self.conv3 = Conv2d(in_channels=batch_size*pow(2,i), out_channels=batch_size*pow(2,i+1), kernel_size=3, padding=1)
        self.conv3_bn = BatchNorm2d(num_features=batch_size*pow(2,i+1))
        i = i + 1
        
        self.conv4 = Conv2d(in_channels=batch_size*pow(2,i), out_channels=batch_size*pow(2,i+1), kernel_size=3, padding=1)
        self.conv4_bn = BatchNorm2d(num_features=batch_size*pow(2,i+1))
        i = i + 1      
        
        self.pool = MaxPool2d(2, 2)
        self.drop1 = Dropout(0.5)
        self.linear1 = Linear(in_features=320000, out_features=1024)
        self.linear2 = Linear(in_features=1024, out_features=512)
        self.linear3 = Linear(in_features=512, out_features=num_classes)
    
    def forward(self, x):
        x = relu(self.conv1_bn(self.conv1(x)))
        x = relu(self.conv2_bn(self.conv2(x)))
        x = self.pool(x)
        x = self.drop1(x)
        
        x = relu(self.conv3_bn(self.conv3(x)))
        x = relu(self.conv4_bn(self.conv4(x)))
        x = self.pool(x)
        x = self.drop1(x)
        
        #print("x.view(self.batch_size, -1)", x.view(self.batch_size, -1).shape)        
        x = x.view(-1, 320000)
        
        x = relu(self.linear1(x))
        x = self.drop1(x)
        
        x = relu(self.linear2(x))
        x = self.drop1(x) 
        
        #x = softmax(self.linear3(x))
        x = self.linear3(x)
        return x

In [7]:
%%time
num_epochs = 5
feature_extract = True

model_ft = MyNet(batch_size, num_classes).double()
print(model_ft)
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)  
criterion = torch.nn.CrossEntropyLoss()
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=False)

MyNet(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.5, inplace=False)
  (linear1): Linear(in_features=320000, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=47,

In [8]:
%%time
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test data: {0}'.format(100 * correct / total))

Accuracy of test data: 13.27683615819209
CPU times: user 2.18 s, sys: 628 ms, total: 2.81 s
Wall time: 2.81 s


In [9]:
%%time
model_name = "model_2.1.model" 
torch.save(model_ft.state_dict(), model_name)
upload_files([model_name])

Upload from local model_2.1.model to data/breakfast-img-data/model_2.1.model
CPU times: user 2.9 s, sys: 5.58 s, total: 8.48 s
Wall time: 39.7 s


In [10]:
%%time
batch_size = 4
num_classes = 47

model_name = "model_2.1.model"
model = MyNet(batch_size, num_classes).double().to(device)
model.load_state_dict(torch.load(model_name, map_location=device)) 

CPU times: user 5.24 s, sys: 3.61 s, total: 8.86 s
Wall time: 5.59 s


<All keys matched successfully>

In [11]:
%%time
test_df = get_test_data()
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  1284 non-null   object
 1   ID       1284 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.2+ KB
None
CPU times: user 396 ms, sys: 1.46 s, total: 1.85 s
Wall time: 1.85 s


In [12]:
%%time
test_data = test_df.sample(frac = 1.0)
test_data['feature'] = test_data.apply(lambda x : extract_or_pad(x.feature), axis = 1)
print(test_data.info())

test_feature = torch.stack(test_data['feature'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        result = np.append(result, predicted.cpu())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284 entries, 426 to 110
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  1284 non-null   object
 1   ID       1284 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 30.1+ KB
None
CPU times: user 9.3 s, sys: 2.58 s, total: 11.9 s
Wall time: 9.32 s


In [13]:
%%time
test_data['label'] = np.transpose(result).astype('int') 
test_data_group = test_data.groupby('ID')['label'].apply(list).reset_index(name='labels')
test_data_group['Category'] = test_data_group.apply(lambda x : np.argmax(np.bincount(x.labels)) + 1, axis = 1)
test_data_group = test_data_group[['ID','Category']]
test_data_group

CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 130 ms


,ID,Category
0,0,2
1,1,2
2,2,2
3,3,2
4,4,24
...,...,...
1279,1279,32
1280,1280,2
1281,1281,32
1282,1282,2


In [14]:
len(test_data_group.Category.unique())

8

In [15]:
submission_name = "submission_2.1_max.csv"
test_data_group.to_csv(submission_name, index=False)

In [16]:
upload_files([submission_name])

Upload from local submission_2.1_max.csv to data/breakfast-img-data/submission_2.1_max.csv


In [17]:
%%time
test_df = get_test_data()
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  1284 non-null   object
 1   ID       1284 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.2+ KB
None
CPU times: user 372 ms, sys: 1.5 s, total: 1.88 s
Wall time: 1.88 s


In [18]:
%%time
test_data = test_df.sample(frac = 1.0)
test_data['feature'] = test_data.apply(lambda x : extract_or_pad(x.feature), axis = 1)
print(test_data.info())

test_feature = torch.stack(test_data['feature'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0]            
        inputs = inputs.to(device)    
        outputs = model(inputs)  
        prop = torch.nn.functional.softmax(outputs).cpu().tolist()
        result = np.append(result, prop)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284 entries, 330 to 1123
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  1284 non-null   object
 1   ID       1284 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 30.1+ KB
None


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


CPU times: user 9.26 s, sys: 2.76 s, total: 12 s
Wall time: 9.5 s


In [19]:
%%time
test_result = pd.DataFrame(result.reshape(-1,47))
test_result['ID'] = test_data['ID'].values
test_data_mean = test_result.groupby('ID')[list(range(47))].agg(['mean'])
test_data_mean.columns = list(range(47))
test_data_mean["Category"] = test_data_mean.idxmax(axis = 1)
test_data_mean['Category'] = test_data_mean.apply(lambda x : x["Category"] + 1, axis = 1).astype('int') 
test_data_mean["ID"] = test_data_mean.index
test_data_group = test_data_mean[['ID','Category']]
print(test_data_group.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284 entries, 0 to 1283
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   ID        1284 non-null   int64
 1   Category  1284 non-null   int64
dtypes: int64(2)
memory usage: 30.1 KB
None
CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 122 ms


In [20]:
len(test_data_group.Category.unique())

9

In [21]:
submission_name = "submission_2.1_mean.csv"
test_data_group.to_csv(submission_name, index=False)

In [22]:
upload_files([submission_name])

Upload from local submission_2.1_mean.csv to data/breakfast-img-data/submission_2.1_mean.csv
